Osnabrück University - A&C: Computational Cognition (Summer Term 2019)

# Exercise Sheet 08: Neural Time Series Data - Part 1

## Introduction

This week's sheet should be solved and handed in at 14:00, on Tuesday 11th of May. If you need help (and Google and other resources were not enough), feel free to contact your tutors. Please push your results to your Github group folder.

In this exercise sheet you will start to analyse neural time series data, i.e. EEG Data. As introduction to 'real' EEG data anaylsis, we will first create and work with artificial signals, in order to better understand the processing procedures. Step by step, we will make our signal more complex, approxiamting real EEG data. Next week, we will then start to take this knowledge in order analyse real EEG Data, recorded by a Neurobiopsychology research group of our institute.

For the following exercises, we will use the signal processing toolbox from [scipy](https://docs.scipy.org/doc/scipy/reference/signal.html). In this documentation, you can find almost all the functions that you need to solve the tasks.

In [1]:
# for more interactive plots, uncomment the follwing line
%matplotlib qt5 

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import random
from scipy import signal

## Assignment 0: Peer review for sheet 01 [3 pts]

Again, you will have to make a peer review of the other groups' solutions. Each group reviews the solutions of two other groups and give points according to the given point distribution considering the correctness of the solution. For this reviews the tutors will give you up to 3 points each week.

| * |Group 1|Group 2|Group 3|Group 4|Group 5|Group 6|Group 7|Group 8|Group 9|Group 10|Group 11|
| ------- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | ------ | ------ |
| check solutions of group: | 9, 5 | 7, 10 | 10, 8  | 8, 3 | 3, 1 | 9, 4  | 6, 4 | 2, 11  | 1, 6  | 11, 7   | 5, 2   |

## Assignment 1: Create and filter simple signals [5 pts]

### a) Simple Sinewave [2 pts]
As a first step we will create a simple signal by combining two single sinusoids. For this:  
- Define a time scale of 1 second, i.e. 1000ms
- Create two sinewaves with a length of 1s: one with a frequency of 10Hz and one with 20Hz
- Then add them together to create your first simple signal
- Plot all three figures

In [3]:
# TODO
t = np.linspace(0, 1, 1000) # define a time scale, 1000 ms
sin1 = np.sin(2 * np.pi * 10 * t) # 10 Hz sinewave
sin2 = np.sin(2 * np.pi * 20 * t) # 20 Hz sinewave
sig = np.concatenate((sin1, sin2)) # adds two sinewaves together

# Plotting of the all three sine waves
# 10 Hz sine wave plot
plt.subplot(3, 1, 1)
plt.plot(sin1)
plt.title('10 Hz')
plt.xlabel('Time (ms)')
plt.ylabel('Amplitude')

# 20 Hz sine wave plot
plt.subplot(3, 1, 2)
plt.plot(sin2)
plt.title('20 Hz')
plt.xlabel('Time (ms)')
plt.ylabel('Amplitude')

# Plot of the two sine waves added together
plt.subplot(3, 1, 3)
plt.plot(sig)
plt.title('10 Hz - 20 Hz concatenated')
plt.xlabel('Time (ms)')
plt.ylabel('Amplitude')

plt.subplots_adjust(hspace=1) # adjusts the distance between the plots for visualization
plt.show()

### b) More complex Signal [2 pts]

Next, we want to achieve something more complex: For this, we select a list of frequencies, that we want our signal to be composed of, and define their amplitueds and their phases. In the following, we have predefined the values that you should use for this.
With the help of these parameters:
- Create a new, a bit more complex signal by combining the resulting sinusoids (you should get 5 sinusoids with the respective *frequ*, *amplit* and *phase*)
- To make it more realistic, create some random Gaussian noise with the same length and add it to your signal
- Then plot both, the clean and the noisy signal

The given Nyquist Frequency *nyq* results from the **nyquist sampling theorem**: It states that in order to prevent distortions of the underyling information, the minimum sampling frequency of a signal (in our case 1000Hz) should be double the frequency of its highest frequency component (i.e. 500Hz).

In [4]:
# set parameters:
srate = 1000 # define sampling rate (signal length = 1s)
nyq = srate/2 # nyquist frequency
frex = [3, 10, 5, 15, 35, 60] # define a list of frequencies
amplit = [5, 15, 10, 5, 7, 1] # define their amplitudes
phases = [np.pi/7, np.pi/8, np.pi, np.pi/2, -np.pi/4, np.pi/3] # and their respective phases

In [ ]:
# create and plot signal (with and gaussian noise)
t = np.linspace(0, 1, 1000, False)  # 1 second
sig = []

# TODO:
#np.sin(2 * np.pi * frex * t + phase) # needs a for-loop?
#noise = np.random.normal(0, 1, 1000)

signal_final = 
signal_noisy = 

### c) Frequency spectrum with FFT [1 pt]
As a next step, we will do a fast Fourier Transform on our signals, in order to get its frequency spectrum. With this, we can check whether it shows all the frequencies that we used to compose it. 
As we will have to compute the frequency spectrum several times, it will be useful to write a function that computes the fourier transform of any signal.
Then compute and plot the FFT of the noisy signal brom 1b).   
If you look at the amplitudes, you will realize that they are half the amplitudes that we predefined when creating the signal. This happens because we are only taking the absolute values of the FT-frequencies.

In [ ]:
def getFT(sig):

# TODO
    
    return FFT

In [ ]:
# compute FFT of the noisy signal
# TODO

## Assignment 2: Filtering in the time-domain vs. filtering in the freuquency-domain [5]
**Convolution theorem:**  
Convolution in the time domain is the same as multiplication in the frequency domain.  

"**Convolution** is used to isolate frequency-band-specific activity and to localize that frequency-band-specific activity in time.  
This is done by convolving wavelets— time-limited sine waves—with EEG data. As the wavelet (the convolution kernel) is dragged along the EEG data (the convolution signal): it reveals when and to what extent the EEG data contain features that look like the wavelet. When convolution is repeated on the same EEG data using wavelets of different frequencies, a time-frequency representation can be formed." (*Mike X Cohen, Analyzing Neural Time Series Data: Theory and Practice*) 

--> to get to know more about convolution I can highly recommend [this](http://mikexcohen.com/lectures.html) video by Mike X Cohen.

### a) Filter in the time domain [3 pts]
To filter our signal in the time domain, we use a windowed sinewave as a filter-kernel. The windowing helps to obtain temporally localized frequency information. We then convolve our constructed wavelet with our signal, extracting the frequency band that we want to work with. 
- First define your pass-band as 25Hz. Ideally everything above this frequency is filtered out, in reality however, we need a transition band of about 10 Hz, or a region between the pass-frequency *f_p* and stop-frequency *f_s*. In this range, frequencies are only attenuated but are not completely excluded. We need this in order to account for the trad-off between precision in the frequency-domain and precision in the time-domain.
- Next, we define the gains of each frequency band: everything outside 0 and our pass-band of 25Hz should be attenuated, i.e. a gain of 0
- Using the function [firwin2](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.signal.firwin2.html) of the signal package and the parameters from above, now construct your filter-kernel *h_win* (the result will be a wavelet with a length of 1s) 
- Plot your kernel as well as it's frequency spectrum. It should look like a step-function, that assigns a gain of 1 to all frequencies in our pass-band of 0-25Hz

In [ ]:
# Create a Low-pass Filter: Windowed 10-Hz transition (0.5 sec)
n = int(srate) 
#n = int(round(srate * 0.5)) + 1
f_p = 25. # define passband
trans_bandwidth = 10.  # 10 Hz transition band
f_s = f_p + trans_bandwidth  # stopband = 40 Hz

# define gains of each frequency band
freq = [0., f_p, f_s, nyq]
gain = [1., 1., 0., 0.]

# compute filter graph with firwin2
h_win = # TODO

# compute frequency spectrum

# Plot both

Now we are ready to convolve our signal with our self-constructed FIR filter. For this, we use the [convolve](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.convolve.html) function of the *signal* package. 
Aftwards, we want to see in the frequency-spectrum whether our filter successfully attenuated the frequency-components above 25Hz. For this, compute and plot the FT of both, the filtered and the unfiltered signal.  
Here you might notice that the huge peak at 35Hz from before filtering is still apparent as a small bumb, since this component was only attenuated, not completely eliminated.   
In order to compare which filtering procedure is faster, record the computation-time of the time-domain convolution with the help of the magic function *%timeit* (you will have to write an extra line for this).

In [ ]:
# TODO
%timeit #TODO

### a) Filter in the frequency domain [2 pts]
Filtering in the frequency domain is computationally much faster and easier. According to the convolution theorem (see above), we just have to multiply the frequency-spectrum of our filter-kernel with the frequency-spectrum of our signal.
After frequency-domain filtering, plot again the frequency response of your filter together with the filtered and the unfiltered signal. 

Again, in order to compare which filtering procedure is faster, record the computation-time of the frequency-domain filtering with the help of the magic function *%timeit*. Then compare it to the computation time of the time-domain convolution. Which one is faster?

In [ ]:
# TODO
%timeit #TODO

In order to compare the filtered and the unfiltered signal in the time-domain, first compute the inverse Fourier-Transform of your convolution result and then plot both signals (unfiltered and filtered) in two separate plots.  

In a second plot, compare your result from filtering in the frequency domain with result from convolution in the time domain (from 2a). Notice how both plots look different, although be basically applied the same filter. This is due to filtering artifacts, which will be further discussed in the lecture.


In [ ]:
# TODO

In [ ]:
# compare convolution result in the frequ. domain with convolution result in the time domain:
# TODO